In [1]:
# imports
from dotenv import load_dotenv

# Load .env File for OpenAI API Key
load_dotenv('../.env')

True

### Set Up Vector Store

In [2]:
# Imports
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Get Embeddings Model
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Initialize ChromaDB as Vector Store
vector_store = Chroma(
    collection_name="test_collection",
    embedding_function=embeddings
)

In [3]:
# Imports
from langchain_text_splitters import CharacterTextSplitter

# Read in State of the Union Address File
with open("../RAG_Docs/2024_state_of_the_union.txt") as f:
    state_of_the_union = f.read()

# Initialize Text Splitter
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

# Create Documents (Chunks) From File
texts = text_splitter.create_documents([state_of_the_union])

# Save Document Chunks to Vector Store
ids = vector_store.add_documents(texts)

# Set Chroma as the Retriever
retriever = vector_store.as_retriever()

In [4]:
# Query the Vector Store to Check Population
results = vector_store.similarity_search(
    'Who invaded Ukraine?',
    k=2
)

# Print Resulting Chunks
for res in results:
    print(f"* {res.page_content} [{res.metadata}]\n\n")

* And yes, my purpose tonight is to both wake up this Congress, and alert the American people that this is no ordinary moment either. 

Not since President Lincoln and the Civil War have freedom and democracy been under assault here at home as they are today. 

What makes our moment rare is that freedom and democracy are under attack, both at home and overseas, at the very same time. 

Overseas, Putin of Russia is on the march, invading Ukraine and sowing chaos throughout Europe and beyond. 

If anybody in this room thinks Putin will stop at Ukraine, I assure you, he will not. 

But Ukraine can stop Putin if we stand with Ukraine and provide the weapons it needs to defend itself. That is all Ukraine is asking. They are not asking for American soldiers. 

In fact, there are no American soldiers at war in Ukraine. And I am determined to keep it that way. 

But now assistance for Ukraine is being blocked by those who want us to walk away from our leadership in the world. [{}]


* But now 

### Advanved RAG: Pre-retrieval Query Rewrting

In [ ]:
# imports
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Pre-retrieval Query Rewriting Function
def query_rewrite(query: str, llm: ChatOpenAI):

    # Rewritten Query Prompt
    query_rewrite_prompt = f"You are a helpful assistant that takes a user's query and turns it into a short statement or paragraph so that it can be used in a semantic similarity search on a vector database to return the most similar chunks of content based on the rewritten query. Please make no comments, just return the rewritten query.\n\nquery: {query}\n\nai: "

    # Invoke LLM
    retrieval_query = llm.invoke(query_rewrite_prompt)

    # Return Generated Retrieval Query
    return retrieval_query

### Advanced RAG: Post-retrieval Reranking

In [6]:
# imports
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

# Flash Rerank Compressor for Post-retrieval Rerank
compressor = FlashrankRerank()

# Update Retriever -> Compression Retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [7]:
# Create Document Parsing Function to String
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

### Create the RAG Prompt

In [8]:
# Create the Prompt Template
prompt_template = """Use the context provided to answer the user's question below. If you do not know the answer based on the context provided, tell the user that you do not know the answer to their question based on the context provided and that you are sorry.
context: {context}

question: {query}

answer: """

# Create Prompt Instance from template
custom_rag_prompt = PromptTemplate.from_template(prompt_template)

### Advanced RAG Pipeline

In [9]:
# Initialize the LLM instance
llm = ChatOpenAI(model="gpt-4o-mini")

In [10]:
# Custom RAG Chain Class
class RAGChain:

    # Chain Constructor
    def __init__(
        self,
        llm: ChatOpenAI,
        retriever: ContextualCompressionRetriever,
        prompt: PromptTemplate
    ):
        self.llm = llm
        self.retriever = retriever
        self.prompt = prompt
    
    # Run Chain Function - same naming convention as LangChain
    def invoke(self, query: str):

        # Advanced RAG: Pre-retrieval Query Rewrite
        retrieval_query = query_rewrite(query, self.llm)
        
        # Retrieval w/ Post-retrieval Reranking
        docs = self.retriever.invoke(retrieval_query.content)

        # Format Docs for Context String
        context = format_docs(docs)

        # Prompt Template
        final_prompt = self.prompt.format(context=context, query=query)

        # LLM Invoke
        return llm.invoke(final_prompt)


In [ ]:
# Initialize Custom RAG Chain
rag_chain = RAGChain(llm, compression_retriever, custom_rag_prompt)

# Invoke the chain
response = rag_chain.invoke('According to the 2024 state of the union address, Who invaded Ukraine and why is this wrong?')

# Print Output
print(response.content)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


According to the 2024 State of the Union address, President Joe Biden stated that Putin of Russia invaded Ukraine. This is considered wrong because it is an assault on freedom and democracy, and it poses a threat not only to Ukraine but also to peace and stability in Europe and beyond. Biden emphasized the need to stand with Ukraine and provide the necessary assistance to defend itself against this aggression.


In [12]:
# Invoke the chain
response = rag_chain.invoke('What is 2+2?')

# Print Output
print(response.content)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


I'm sorry, but I do not know the answer to your question based on the context provided.
